<a href="https://colab.research.google.com/github/Enkrumah14/mannyNkrumahGenAi/blob/main/Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import requests
import re
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Download and combine Shakespeare texts
urls = [
    "https://www.gutenberg.org/files/1041/1041-0.txt",  # Hamlet
    "https://www.gutenberg.org/files/152/152-0.txt",    # Macbeth
    "https://www.gutenberg.org/files/1112/1112-0.txt"   # Othello
]

def download_and_combine_texts(urls):
    combined_text = ""
    for url in urls:
        response = requests.get(url)
        text = response.text
        text = re.sub(r'\[.*?\]', '', text)  # Remove content in brackets
        text = re.sub(r'\n\n+', '\n', text)  # Remove multiple newlines
        combined_text += text + "\n\n"
    return combined_text

# Get and preprocess text
text = download_and_combine_texts(urls)
print(f"Total characters: {len(text)}")

# Create character-level sequences
chars = sorted(list(set(text)))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

# Parameters
SEQ_LENGTH = 100
BATCH_SIZE = 64
BUFFER_SIZE = 10000
EMBEDDING_DIM = 256
N_UNITS = 512
VOCAB_SIZE = len(chars)

def create_sequences(text, seq_length):
    input_sequences = []
    target_sequences = []

    for i in range(0, len(text) - seq_length):
        seq = text[i:i + seq_length]
        target = text[i + 1:i + seq_length + 1]  # Next sequence shifted by one character
        input_sequences.append([char_to_idx[char] for char in seq])
        target_sequences.append([char_to_idx[char] for char in target])

    x = np.array(input_sequences)  # Shape: (num_sequences, seq_length)
    y = np.array(target_sequences)  # Shape: (num_sequences, seq_length)
    return x, y

x, y = create_sequences(text, SEQ_LENGTH)
print(f"Input shape: {x.shape}, Target shape: {y.shape}")

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

def create_model():
    inputs = layers.Input(shape=(SEQ_LENGTH,))
    x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
    x = layers.LSTM(N_UNITS, return_sequences=True)(x)
    x = layers.Dropout(0.2)(x)  # Prevent overfitting
    x = layers.LSTM(N_UNITS, return_sequences=True)(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(VOCAB_SIZE, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = create_model()
model.summary()

def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    input_eval = [char_to_idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        text_generated.append(idx_to_char[predicted_id])
        input_eval = tf.concat([input_eval[:, 1:],
                                tf.expand_dims([predicted_id], 0)], axis=-1)

    return start_string + ''.join(text_generated)

# Training parameters
EPOCHS = 20
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

# Train the model
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Generate text with different temperatures
prompts = [
    "To be, or not to be",
    "Shall I compare thee to a summer's day?",
    "All the world's a stage"
]

temperatures = [0.2, 0.5, 1.0]

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    for temp in temperatures:
        print(f"\nTemperature: {temp}")
        generated_text = generate_text(model, prompt, num_generate=500, temperature=temp)
        print(generated_text)


Total characters: 245274
Input shape: (245174, 100), Target shape: (245174, 100)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 100, 256)          24064     
                                                                 
 lstm_2 (LSTM)               (None, 100, 512)          1574912   
                                                                 
 dropout_2 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_3 (LSTM)               (None, 100, 512)          2099200   
                                                                 
 dropout_3 (Dropout)         (None, 100, 512)          0         
                                            